In [3]:
import numpy as np
from dataset_welch import create_dataset
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest
plt.rcParams['figure.dpi'] = 200
plt.rcParams.update({
    "text.usetex": True
    })
plt.rcParams.update({'font.size': 18})
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.linear_model import ARDRegression, LinearRegression, BayesianRidge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

In [4]:
taguchi_array = np.array(loadmat('taguchiQ3N20.mat')['ans'])
orthogonal_array = taguchi_array[:, np.random.permutation(taguchi_array.shape[1])]

x_train, y_train = create_dataset(taguchi_array)
x_test, y_test = create_dataset(orthogonal_array)
x_test, y_test = x_test[:20, :], y_test[:20] # reduce test size

x = np.array([i for i in range(20)])

def get_model_results(idx, X, y, x_test, y_test):

    olr = LinearRegression().fit(X[:, idx], y)
    brr = BayesianRidge(compute_score=True, n_iter=30).fit(X[:, idx], y)
    ard = ARDRegression(compute_score=True, n_iter=30).fit(X[:, idx], y)

    models = [olr, brr, ard]

    rmse_models = []
    r2_models = []

    for model in models:
        y_pred = model.predict(x_test[:, idx])
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        test_r2 = BestFitRate(y_test, y_pred)
        rmse_models.append(test_rmse)
        r2_models.append(test_r2)
    print(['LR', 'BR', 'ARD'])
    print(r2_models)
    return rmse_models, r2_models

def BestFitRate(y_target,y_est):
    
    if len(y_target)==len(y_est):
        N = len(y_target)
    else:
        print("Dimensions don't match!")
        return None
    
    y_target = np.array(y_target).reshape((N,-1))
    y_est = np.array(y_est).reshape((N,-1))
    
    y_target = y_target.astype(float)
    y_est = y_est.astype(float)
    
    
    e_pred = np.linalg.norm(y_target-y_est,axis=1)
    e_mean = np.linalg.norm(y_target-np.mean(y_target),axis=1)
    
    BFR = 1-sum(e_pred**2) / sum(e_mean**2) 
    
    BFR = BFR*100
    
    if BFR<0:
        BFR = 0
        
    return BFR

In [5]:
class FeatureImportanceEnsemble:
    def __init__(self, x_train, y_train, n_features_to_select=3):

        self.x_train = x_train
        self.y_train = y_train
        self.n_features_to_select = n_features_to_select

        self.alpha = 0.1 # for lasso, ridge, elnet

        self.fi1 = self.loo(self.get_dt)
        self.fi2 = self.loo(self.get_f_test)
        self.fi3 = self.loo(self.get_p_vals)
        self.fi4 = self.loo(self.get_lasso_fi)
        self.fi5 = self.loo(self.get_ridge_fi)
        self.fi6 = self.loo(self.get_elnet_fi)
        self.fi7 = self.loo(self.get_ard_precision)
        self.fi8 = self.loo(self.get_rf_fi)
        self.fi9 = self.loo(self.get_lr_fi)
        self.fi10 = self.loo(self.get_xgboost_fi)

        self.fi11 = np.random.uniform(0, 1, size=(1, 20)) # expert knowledge
        
        self.inner_ensemble_results = self.ensemble_inner_decision_process()
        self.outer_ensemble_results = np.argsort(-self.inner_ensemble_results.mean(axis=0))
        self.inner_ensemble_features = self.fi_single_choosen_features()
        self.outer_ensemble_features = self.ensemble_outer_decision_process()
        #self.ensemble = self.get_ensemble_results()

    def loo(self, fi_model_function):
        # insert feature importance algorithm
        # do loo
        # return feature importance of every run
        x_train = self.x_train
        y_train = self.y_train

        loo = LeaveOneOut()

        fis = []
        for train, test in loo.split(x_train, y_train):
            fis.append(fi_model_function(x_train[train], y_train[train]))

        fis = np.vstack(fis)
        return fis 

    def get_dt(self, x_train, y_train): # Gini Importance from DecisionTreeRegressior
        model = DecisionTreeRegressor()
        model.fit(x_train, y_train)
        return model.feature_importances_

    def get_f_test(self, x_train, y_train):
        f_test, p_values = f_regression(x_train, y_train)
        f_test /= np.max(f_test)
        return f_test

    def get_p_vals(self, x_train, y_train):
        f_test, p_values = f_regression(x_train, y_train)
        p_values = -np.log10(p_values)
        p_values /= p_values.max()
        return p_values

    def get_lasso_fi(self, x_train, y_train):
        lasso = Lasso(alpha=self.alpha, fit_intercept=False).fit(x_train, y_train)
        lasso_importance = lasso.coef_
        lasso_importance /= np.max(lasso_importance)
        return lasso_importance

    def get_ridge_fi(self, x_train, y_train):
        ridge = Ridge(alpha=self.alpha, fit_intercept=False).fit(x_train, y_train)
        ridge_importance = ridge.coef_
        ridge_importance /= np.max(ridge_importance)
        return ridge_importance
    
    def get_elnet_fi(self, x_train, y_train):
        el_net = ElasticNet(alpha=self.alpha, fit_intercept=False).fit(x_train, y_train)
        el_net_importance = el_net.coef_
        el_net_importance /= np.max(el_net_importance)
        return el_net_importance

    def get_ard_precision(self, x_train, y_train):
        ard = ARDRegression(compute_score=True, n_iter=30).fit(x_train, y_train)
        prec = 1/ard.lambda_
        prec_minmax = (prec - prec.min())/(prec.max()-prec.min())
        return prec_minmax

    def get_rf_fi(self, x_train, y_train):
        rf = RandomForestRegressor()
        rf.fit(x_train, y_train)
        return rf.feature_importances_
    
    def get_lr_fi(self, x_train, y_train):
        lr = Lasso(alpha=0, fit_intercept=False).fit(x_train, y_train)
        lr_importance = lr.coef_
        lr_importance /= np.max(lr_importance)
        return lr_importance

    def get_xgboost_fi(self, x_train, y_train):
        xgbr = xgb.XGBRegressor(verbosity=0) 
        xgbr.fit(x_train, y_train)
        return xgbr.feature_importances_
        
    def ensemble_inner_decision_process(self):
        fi1 = self.fi1
        fi2 = self.fi2
        fi3 = self.fi3
        fi4 = self.fi4
        fi5 = self.fi5
        fi6 = self.fi6
        fi7 = self.fi7
        fi8 = self.fi8
        fi9 = self.fi9
        fi10 = self.fi10
        fi11 = self.fi11
        
        fi_list = [fi1, fi2, fi3, fi4, fi5, fi6, fi7, fi8, fi9, fi10, fi11]
        mean_values = []
        for fi in fi_list:
            mean_values.append(fi.mean(axis=0))
        mean_values = np.vstack(mean_values)
        return mean_values
    
    def ensemble_outer_decision_process(self):
        outer_ensemble_results = self.inner_ensemble_results.mean(axis=0)
        n_features_to_select = self.n_features_to_select

        choosen_features = np.argpartition(outer_ensemble_results, -n_features_to_select)[-n_features_to_select:] # get the n_features_to_select largest elements
        return choosen_features

    def fi_single_choosen_features(self):
        n_features_to_select = self.n_features_to_select
        fi_single_choosen_features = []
        for res in self.inner_ensemble_results:
            choosen_features = np.argpartition(res, -n_features_to_select)[-n_features_to_select:]

            fi_single_choosen_features.append(choosen_features)
        fi_single_choosen_features = np.vstack(fi_single_choosen_features)
        return fi_single_choosen_features

    def get_idx(self, fi):
        return np.argsort(-fi.mean(axis=0)).tolist()

In [6]:
FIEnsemble = FeatureImportanceEnsemble(x_train, y_train, n_features_to_select=3)

features_prio_idx = FIEnsemble.outer_ensemble_results.tolist()
features_prio_idx

/tmp/ipykernel_19881/3745804635.py:90: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lr = Lasso(alpha=0, fit_intercept=False).fit(x_train, y_train)
/home/hoschangn/anaconda3/envs/AutoML/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/hoschangn/anaconda3/envs/AutoML/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.724e+02, tolerance: 1.422e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV inst

[11, 18, 16, 14, 9, 7, 5, 15, 17, 6, 4, 19, 8, 13, 1, 2, 10, 0, 3, 12]

In [13]:
print(FIEnsemble.get_idx(FIEnsemble.fi1))
print(FIEnsemble.get_idx(FIEnsemble.fi2))
print(FIEnsemble.get_idx(FIEnsemble.fi3))
print(FIEnsemble.get_idx(FIEnsemble.fi4))
print(FIEnsemble.get_idx(FIEnsemble.fi5))
print(FIEnsemble.get_idx(FIEnsemble.fi6))
print(FIEnsemble.get_idx(FIEnsemble.fi7))
print(FIEnsemble.get_idx(FIEnsemble.fi8))
print(FIEnsemble.get_idx(FIEnsemble.fi9))
print(FIEnsemble.get_idx(FIEnsemble.fi10))
print(FIEnsemble.get_idx(FIEnsemble.fi11))

[11, 18, 14, 16, 0, 9, 7, 10, 1, 6, 3, 12, 17, 13, 4, 5, 15, 2, 8, 19]
[11, 18, 14, 16, 7, 9, 4, 8, 2, 10, 1, 0, 3, 13, 5, 6, 19, 15, 17, 12]
[11, 18, 14, 16, 7, 9, 4, 8, 2, 10, 1, 13, 5, 6, 17, 3, 0, 15, 19, 12]
[11, 18, 14, 16, 7, 9, 4, 17, 15, 13, 12, 0, 8, 6, 5, 3, 2, 1, 10, 19]
[11, 18, 14, 16, 7, 9, 4, 2, 1, 6, 0, 3, 19, 12, 15, 17, 5, 13, 10, 8]
[11, 18, 14, 16, 7, 9, 4, 2, 17, 15, 13, 12, 0, 6, 5, 3, 1, 19, 10, 8]
[11, 18, 14, 16, 7, 9, 4, 8, 17, 5, 10, 2, 1, 13, 6, 3, 15, 19, 12, 0]
[11, 18, 14, 16, 0, 7, 9, 10, 19, 4, 5, 12, 6, 13, 1, 15, 17, 3, 8, 2]
[11, 18, 14, 16, 7, 9, 4, 2, 1, 6, 0, 3, 19, 12, 15, 17, 5, 13, 10, 8]
[11, 18, 16, 14, 12, 7, 0, 10, 6, 13, 1, 5, 2, 4, 15, 3, 17, 9, 8, 19]
[5, 15, 17, 16, 6, 9, 19, 8, 13, 7, 1, 18, 2, 10, 11, 0, 14, 4, 3, 12]


In [8]:
for i in range(19):
    get_model_results(features_prio_idx[0:i+1], x_train, y_train, x_test, y_test)

['LR', 'BR', 'ARD']
[42.64232053114354, 42.37783892056704, 42.37783892056706]
['LR', 'BR', 'ARD']
[65.53715508426622, 65.33461483574008, 65.33688059819292]
['LR', 'BR', 'ARD']
[54.795991775111474, 54.93567700501385, 55.63711153404014]
['LR', 'BR', 'ARD']
[47.88109924687913, 48.437089432623836, 49.24627673348033]
['LR', 'BR', 'ARD']
[43.87596323736385, 44.51743499530302, 45.30695826607012]
['LR', 'BR', 'ARD']
[42.14184530867545, 42.764836640532536, 43.405350392458075]
['LR', 'BR', 'ARD']
[42.15837079532111, 42.893210603657096, 43.40558368927548]
['LR', 'BR', 'ARD']
[42.158370795321055, 43.007139902406564, 43.405584414157204]
['LR', 'BR', 'ARD']
[42.18607365479311, 43.15071552443398, 43.40595783060005]
['LR', 'BR', 'ARD']
[42.062419341505255, 43.15042464436816, 43.40465889106269]
['LR', 'BR', 'ARD']
[38.620380689673716, 39.856423465377446, 40.47120070518473]
['LR', 'BR', 'ARD']
[38.620380689673716, 39.98455972175022, 40.471202859260046]
['LR', 'BR', 'ARD']
[38.49569006367346, 39.98908955

In [9]:
prio_idx = FIEnsemble.get_idx(FIEnsemble.fi1)
for i in range(19):
    get_model_results(prio_idx[0:i+1], x_train, y_train, x_test, y_test)

['LR', 'BR', 'ARD']
[42.64232053114354, 42.37783892056704, 42.37783892056706]
['LR', 'BR', 'ARD']
[65.53715508426622, 65.33461483574008, 65.33688059819292]
['LR', 'BR', 'ARD']
[64.12715568419327, 64.23535284312054, 64.44513769975768]
['LR', 'BR', 'ARD']
[47.88109924687917, 48.43708943262386, 49.24627673348034]
['LR', 'BR', 'ARD']
[47.881099246879145, 48.580543160114544, 49.2462768005135]
['LR', 'BR', 'ARD']
[43.87596323736381, 44.652517114047185, 45.30695827051671]
['LR', 'BR', 'ARD']
[42.141845308675464, 42.87634679630044, 43.40535039430503]
['LR', 'BR', 'ARD']
[42.43248784480155, 43.2650254070649, 43.406336491413946]
['LR', 'BR', 'ARD']
[42.452658931524645, 43.40727441410285, 43.406519807439636]
['LR', 'BR', 'ARD']
[42.328079339029344, 43.40483236428252, 43.405101133910094]
['LR', 'BR', 'ARD']
[42.32807933902934, 43.5254646268287, 43.40509884665946]
['LR', 'BR', 'ARD']
[42.32807933902932, 43.64873528609154, 43.40509657196196]
['LR', 'BR', 'ARD']
[42.356115299016054, 43.80199367234789